In [1]:
import numpy as np
import pandas as pd
import pickle
import datetime

## Update Local Path

In [2]:
#path = "../../../Google Drive/Data_science/NYU/Machine Learning/ML Project (Collisions)/" #Joe
path = "../../../../Google Drive/ML Project (Collisions)/" # Joyce
# path = "" # Lucas

In [3]:
#Bin size
steps = [0.001, 0.002, 0.005, .03, .06, .1, .3]
#Number of days before time to count complaints
days = 90

In [14]:
df = pd.read_csv(path + "NYPD_Motor_Vehicle_Collisions.csv", parse_dates=[['DATE', 'TIME']], infer_datetime_format=True)
df = df[df.LATITUDE.notnull()]
df = df.reset_index()

In [15]:
for step in steps:
    to_bin = lambda x: np.floor(x / step) * step
    df["lat_bin_"+str(step)] = df.LATITUDE.map(to_bin)
    df["long_bin_"+str(step)] = df.LONGITUDE.map(to_bin)

## Join 311 data

In [16]:
complaints = pd.read_csv(path + "Google_Map_-_311_NYPD_Traffic_Complaints_-_2010-Present.csv")
complaints['Created Date'] = pd.to_datetime(complaints['Created Date'], infer_datetime_format=True)
complaints = complaints[complaints['Created Date'].dt.date < datetime.date(2015, 9, 12)]

for step in steps:
    to_bin = lambda x: np.floor(x / step) * step
    complaints["lat_bin_"+str(step)] = complaints.Latitude.map(to_bin)
    complaints["long_bin_"+str(step)] = complaints.Longitude.map(to_bin)

In [17]:
complaints['Descriptor'].unique()

array(['Congestion/Gridlock', 'Truck Route Violation', 'Drag Racing',
       'Chronic Stoplight Violation', 'Chronic Speeding'], dtype=object)

In [18]:
for step in steps:
    grouped = pd.DataFrame(complaints.groupby(['lat_bin_'+str(step), 'long_bin_'+str(step), 'Descriptor'])['Unique Key'].count().reset_index(level=2))
    
    for comp_type in grouped.Descriptor.unique():
        renamed = grouped[grouped['Descriptor'] == comp_type]\
        .rename(columns={'Unique Key': comp_type+' '+str(step)+' bin' }).drop(['Descriptor'], axis=1)
        newdf = df.join(renamed, on=["lat_bin_"+str(step), "long_bin_"+str(step)], how='left')
        df[comp_type+' '+str(step)+' bin'] = newdf[comp_type+' '+str(step)+' bin'].fillna(value=0)

In [19]:
df.set_index(['UNIQUE KEY'], inplace=True)

In [20]:
drop_columns = [i for i in df.columns.values if ' bin' not in i]

In [21]:
df_joined = df.drop(drop_columns, axis=1)

In [22]:
df_joined

,Congestion/Gridlock 0.001 bin,Chronic Speeding 0.001 bin,Drag Racing 0.001 bin,Chronic Stoplight Violation 0.001 bin,Truck Route Violation 0.001 bin,Congestion/Gridlock 0.002 bin,Chronic Speeding 0.002 bin,Drag Racing 0.002 bin,Chronic Stoplight Violation 0.002 bin,Truck Route Violation 0.002 bin,...,Chronic Speeding 0.1 bin,Chronic Stoplight Violation 0.1 bin,Congestion/Gridlock 0.1 bin,Drag Racing 0.1 bin,Truck Route Violation 0.1 bin,Chronic Speeding 0.3 bin,Chronic Stoplight Violation 0.3 bin,Congestion/Gridlock 0.3 bin,Drag Racing 0.3 bin,Truck Route Violation 0.3 bin
UNIQUE KEY,,,,,,,,,,,,,,,,,,,,,
3628188,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,198.0,267.0,1795.0,93.0,238.0,775.0,1056.0,10121.0,405.0,1184.0
3628414,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,221.0,391.0,5004.0,98.0,371.0,775.0,1056.0,10121.0,405.0,1184.0
3628292,7.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,2.0,0.0,...,221.0,391.0,5004.0,98.0,371.0,775.0,1056.0,10121.0,405.0,1184.0
3627838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,198.0,267.0,1795.0,93.0,238.0,775.0,1056.0,10121.0,405.0,1184.0
3284922,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,221.0,391.0,5004.0,98.0,371.0,775.0,1056.0,10121.0,405.0,1184.0
3627880,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,130.0,167.0,847.0,76.0,146.0,775.0,1056.0,10121.0,405.0,1184.0
3627532,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,101.0,96.0,729.0,80.0,79.0,172.0,197.0,1536.0,227.0,172.0
3628782,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,198.0,267.0,1795.0,93.0,238.0,775.0,1056.0,10121.0,405.0,1184.0
3517274,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,221.0,391.0,5004.0,98.0,371.0,775.0,1056.0,10121.0,405.0,1184.0


In [23]:
with open(path+"data_for_joining/311_joined.pkl", 'wb') as outfile:
    pickle.dump(df_joined, outfile, pickle.HIGHEST_PROTOCOL)

## For missing lat-longs

In [4]:
df = pd.read_csv(path+'NYPD_Motor_Vehicle_Collisions_LATLONGCLEANED.csv',\
                 usecols=['LATITUDE', 'LONGITUDE', 'Lat/Long Infered', 'UNIQUE KEY'])
df = df [(df['Lat/Long Infered'] != 'Original') & (df['Lat/Long Infered'] != 'Unable_to_infer')]

In [5]:
for step in steps:
    to_bin = lambda x: np.floor(x / step) * step
    df["lat_bin_"+str(step)] = df.LATITUDE.map(to_bin)
    df["long_bin_"+str(step)] = df.LONGITUDE.map(to_bin)

In [7]:
complaints = pd.read_csv(path+"Google_Map_-_311_NYPD_Traffic_Complaints_-_2010-Present.csv")
complaints['Created Date'] = pd.to_datetime(complaints['Created Date'], infer_datetime_format=True)
complaints = complaints[complaints['Created Date'].dt.date < datetime.date(2015, 9, 12)]

for step in steps:
    to_bin = lambda x: np.floor(x / step) * step
    complaints["lat_bin_"+str(step)] = complaints.Latitude.map(to_bin)
    complaints["long_bin_"+str(step)] = complaints.Longitude.map(to_bin)

In [8]:
for step in steps:
    grouped = pd.DataFrame(complaints.groupby(['lat_bin_'+str(step), 'long_bin_'+str(step), 'Descriptor'])['Unique Key'].count().reset_index(level=2))
    
    for comp_type in grouped.Descriptor.unique():
        renamed = grouped[grouped['Descriptor'] == comp_type]\
        .rename(columns={'Unique Key': comp_type+' '+str(step)+' bin' }).drop(['Descriptor'], axis=1)
        newdf = df.join(renamed, on=["lat_bin_"+str(step), "long_bin_"+str(step)], how='left')
        df[comp_type+' '+str(step)+' bin'] = newdf[comp_type+' '+str(step)+' bin'].fillna(value=0)
        print ('Finished %s, %s' % (step, comp_type))

df.set_index(['UNIQUE KEY'], inplace=True)

Finished 0.001, Congestion/Gridlock
Finished 0.001, Chronic Speeding
Finished 0.001, Drag Racing
Finished 0.001, Chronic Stoplight Violation
Finished 0.001, Truck Route Violation
Finished 0.002, Congestion/Gridlock
Finished 0.002, Chronic Speeding
Finished 0.002, Drag Racing
Finished 0.002, Chronic Stoplight Violation
Finished 0.002, Truck Route Violation
Finished 0.005, Congestion/Gridlock
Finished 0.005, Chronic Speeding
Finished 0.005, Chronic Stoplight Violation
Finished 0.005, Drag Racing
Finished 0.005, Truck Route Violation
Finished 0.03, Chronic Speeding
Finished 0.03, Chronic Stoplight Violation
Finished 0.03, Congestion/Gridlock
Finished 0.03, Drag Racing
Finished 0.03, Truck Route Violation
Finished 0.06, Chronic Speeding
Finished 0.06, Chronic Stoplight Violation
Finished 0.06, Congestion/Gridlock
Finished 0.06, Drag Racing
Finished 0.06, Truck Route Violation
Finished 0.1, Chronic Speeding
Finished 0.1, Chronic Stoplight Violation
Finished 0.1, Congestion/Gridlock
Finished

In [10]:
drop_columns = [i for i in df.columns.values if ' bin' not in i]

In [11]:
df_joined = df.drop(drop_columns, axis=1)

In [13]:
with open(path+"inferred_lat_long_data/311_joined.pkl", 'wb') as outfile:
    pickle.dump(df_joined, outfile, pickle.HIGHEST_PROTOCOL)

## Only for re-running 90 days before

In [ ]:
complaints['Created Date'] = pd.to_datetime(complaints['Created Date'], infer_datetime_format=True)

In [ ]:
def time_before(lat, lon, start_date, days):
    within = complaints[(complaints['Created Date'] < start_date) & (complaints['Created Date'] > start_date - datetime.timedelta(days))]
    groupedtime = pd.DataFrame(within.groupby(['lat_bin', 'long_bin', 'Descriptor'])['Unique Key'].count().reset_index(level=2))

    counts = []
    
    for comp_type in grouped.Descriptor.unique():
        try:
            value = int(groupedtime[groupedtime['Descriptor'] == comp_type].loc[lat,lon].values[1])
        except:
            value = 0
            
        counts.append(value)    
    return counts

In [ ]:
before_counts = []

for idx, row in df.iterrows():
    counts = time_before(row['lat_bin'], row['long_bin'], row['DATE_TIME'], days)
    before_counts.append(counts)
    
    if idx % 100 == 0:
        print '%s/%s rows completed' % (idx, df.shape[0])

In [ ]:
j = 0

for comp_type in grouped.Descriptor.unique():
    df[comp_type+' count '+days+' days before'] = [i[j] for i in before_counts]
    j += 1

In [ ]:
df_311_joined = df.set_index(['UNIQUE KEY'].df.drop(df.columns.values[:30], axis=1)

In [ ]:
with open('311_joined.pkl', 'wb') as output:
    pickle.dump(df_311_joined, output, pickle.HIGHEST_PROTOCOL)